# Basic data manipulation in ML

Before starting any ML project, one must firt look at the data at hand, to discover what kinds of features we have, clean it, place it in the right format, discover relations between features, etc.

This is called **Data Munging**

In [ ]:
# Import first some useful tools

import numpy as np  # numpy for mathematical manipulation of arrays
import matplotlib  # matplotlib for graphics
from matplotlib import pyplot as plt
import seaborn as sb  # Use also seaborn for plots
import pandas as pd  # pandas for database manipulations

import sklearn  # Scikit-learn, many tools for ML

# Diabetes dataset

Dataset of medical characteristics of several individuals, toghether with a diagnostic of diabetes. Can be used to predict the possibility of diabetes previous to the onset of symptoms

In [ ]:
# load  diabetes database
# https://www.kaggle.com/datasets/mathchi/diabetes-data-set?select=diabetes.csv
# Gives some helth variables of patients (glucose concentration, blood pressure, age, etc) as features and 0 or 1 as label
# indicating if the individuals is diabetic or not

# We read using a Pandas, a very powerful tool for data analysis
# Pandas operates with dataframes, that allow for many manipulations and transformations of data

dataset = pd.read_csv('data/diabetes_database.csv')

In [ ]:
# look at the first rows of data
dataset.head()

The first columns represent the features that characterize each datapoint. The last column represents the label, whether the individual develops diabetes or not.

In [ ]:
# description of the data
dataset.describe()

In [ ]:
# check if there are problems in the dataset, such as empty cells,
# that must be fixed before applying any algorithms

dataset.isnull().sum()

In [ ]:
# First of all, study the distribution of the feature components, to see if there is any strange behavior or possible outliers
# In terms of a boxplot
# A box plot (aka box and whisker plot) uses boxes and lines to depict the distributions of one or more groups of numeric data. 
# Box limits indicate the range of the central 50% of the data, with a central line marking the median value.

dataset.boxplot(figsize=(12, 6))
plt.show()

In [ ]:
# A histogram for each feature component
_ = dataset.hist(figsize=(12, 10))
plt.tight_layout()
plt.show()

We might have some problems, since some components have a much wider range of variation than other. We should probably need to perform a data rescaling, using standardization or normalization

Before that, let us see if there are correlations between the features of the different datapoints

In [ ]:
correlations = dataset.corr()
plt.figure(figsize = (7, 7))
sb.heatmap(correlations, fmt = ".3f", annot = True, cmap = "YlGnBu")
plt.show()

Features can be considered as independent

In [ ]:
# Extract features and labels as numpy arrays for convenience

X = dataset.drop("Outcome", axis=1).to_numpy()
y = dataset.Outcome.to_numpy()

In [ ]:
# Size of vectors
X.shape, y.shape

768 datapoints, feature vectors of 8 dimensions

Let us now rescale the data, using a sklearn tool

In [ ]:
# define the scaler
scaler = sklearn.preprocessing.StandardScaler()

In [ ]:
# train the scaler
scaler.fit(X)

In [ ]:
# Scale the data

X = scaler.transform(X)

In [ ]:
# Before we apply any algorithm we must choose our training, validation and test sets.  
# Work only with training and test

# The module scikit-learn has tools for this purpose

# We use a fraction 75% / 25%

X_train, X_test, y_train, y_test =  sklearn.model_selection.train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

# Fashion MNIST dataset

Dataset of Zalando's article images, labeled according to the class of article

In [ ]:
# You can download the dataset from 
# https://www.kaggle.com/datasets/zalando-research/fashionmnist/data
# It already comes alredy divided in train and test sets

train = pd.read_csv('data/fashion-mnist_train.csv')
test = pd.read_csv('data/fashion-mnist_test.csv')

In [ ]:
# look at the first rows of data
train.head()

Each datapoint is now an image, represented in raster format. Each one is a vector of dimension 784 (discarding the label, which corresponds to the type of item). This correspods to an image of size $28 \times 28$ pixels

In [ ]:
# The names corresponding to each label

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                   "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# check if there are problems in the dataset, such as empty cells,
# that must be fixed before applying any algorithms

train.isnull().sum()

In [ ]:
# Extract the sets as numpy arrays
X_train = train.drop("label", axis=1).to_numpy()
y_train = train.label.to_numpy()

X_test = test.drop("label", axis=1).to_numpy()
y_test = test.label.to_numpy()

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.max()

The features correspond to the color of the pixel, in a greyscale format. For convinience we normalize the number in the range $[0,1]$

In [ ]:
X_train = X_train/X_train.max()
X_test = X_test/X_train.max()

Images of size 28 x 28. Let us visualize them first

In [ ]:
# Plot the first images of the test set

n = 10

fig, axes = plt.subplots(nrows=n, ncols=n, figsize=(18*n/20, 18*n/20))

axes = axes.flatten()

for i in range(n*n):
    value = class_names[y_test[i]]

    axes[i].set_title(value)

    # To plot them, we have to reshape the 784 vector into a 
    axes[i].imshow(X_test[i].reshape(28, 28), cmap = matplotlib.cm.binary, interpolation="nearest")

    axes[i].axis("off")

plt.tight_layout()
plt.show()

The dimensionality of the feature vectors is too large. We can try to reduce it using Principal Component Analysis (PCA)

In [ ]:
# perform diffferent PCAs measuring the explained variance as a function of the number of components

explained = []
components = np.arange(1, 200, 25)

for v in components:
    pca = sklearn.decomposition.PCA(n_components=v)
    pca.fit(X_train)    

    print(v, pca.explained_variance_ratio_.sum())
    
    explained.append(pca.explained_variance_ratio_.sum())


In [ ]:
# plot the variance explained as a function of the number of components

plt.plot(components, explained, "o-", markersize=7)

plt.xlabel("Number of components")
plt.ylabel("Variance explained")

plt.show()

125 components should be enough

In [ ]:
C = 125
pca = sklearn.decomposition.PCA(n_components=C)
pca.fit(X_train)

print(f'Total number of components used after PCA : {pca.n_components_}')

pca.explained_variance_ratio_.sum()

In [ ]:
# Transform the train and test sets

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
X_train_pca.shape

In [ ]:
# We can invert the PCA transformation, recovering a 28 x 28 image only from the preserved 125 components, and descarding the 648 others

X_train_pca_recovered = pca.inverse_transform(X_train_pca)

In [ ]:
X_train_pca_recovered.shape

In [ ]:
# Plot the inverted figures

n = 10

fig, axes = plt.subplots(nrows=n, ncols=n, figsize=(15*n/20, 15*n/20))

axes = axes.flatten()

for i in range(n*n):

    axes[i].imshow(X_train_pca_recovered[i,:].reshape(28,28), cmap = matplotlib.cm.binary, interpolation="nearest")
    axes[i].axis("off")

plt.show()

Quite good good approximation; we can work with the 125 components of the PCA transformed data, and see how it works

We can make a first visual study applying the tSNE dimensional reduction

In [ ]:
# Use tSNE for visualization Laptop

# Use the first 10000 points of the train dataset
X_tot_pca = X_train_pca[:10000]

In [ ]:
# Define and train the tSNE algorithm for a dimensional reduction to 2 dimensions
X_tsne = sklearn.manifold.TSNE(n_components=2, random_state=12).fit_transform( X_tot_pca ) 

In [ ]:
# Plot the points of the dimensional reduction as a scatter plot

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(9, 7))

# every point has a color given by its label (the real value of the digit)
colors = y_train[:10000].astype(float)/y_train[:10000].astype(float).max()

c_ticks = np.sort(np.array(list(set(colors)), dtype=float))

plt.scatter(X_tsne[:,0], X_tsne[:,1], s=5, c=colors, cmap='jet')

cb = plt.colorbar(ticks=c_ticks)

_ = cb.ax.set_yticklabels(class_names)

plt.show()

Some misclassified points but a very good result in average

# Higgs Boson dataset

Dataset of events in particle accelerator, providing different measures and a label, indicating whether the event corresponds to the observation of a Higgs boson

In [ ]:
# Read the data
# https://www.neuraldesigner.com/wp-content/uploads/2023/10/higgs.csv

bosons = pd.read_csv("data/higgs.csv", delimiter=";")

In [ ]:
bosons.head()

In [ ]:
bosons.describe()

In [ ]:
bosons.boxplot(figsize=(12, 6))
plt.show()

The values are not very different, so we can probably do without rescaling the data

In [ ]:
bosons.hist(bins=50, figsize=(16, 10))

plt.tight_layout()
plt.show()

The event takes a value close to zero if it is negative (no Higgs Boson), and close to 1 if it is positive

For correlations between features:

In [ ]:
correlations = bosons.corr()
plt.figure(figsize = (20, 20))
sb.heatmap(correlations, fmt = ".3f", annot = True, cmap = "YlGnBu")
plt.show()

Cross-correlations are generally weak, some features can be considered as independent, but some pairs are highly correlated. They might be removed, to reduce dimensionality


Let us define features and labels, and make the split in train and test sets

We can do it directly with the `pandas` library

In [ ]:
bosons_train, bosons_test = sklearn.model_selection.train_test_split(bosons, test_size=0.2, random_state=423)

In [ ]:
# Define now the features and labels

X_train = bosons_train.drop("Event", axis=1).to_numpy()
y_train = bosons_train["Event"].copy().to_numpy()

X_test = bosons_test.drop("Event", axis=1).to_numpy()
y_test = bosons_test["Event"].copy().to_numpy()

In [ ]:
X_train.shape, y_train.shape

# 8009 datapoints with 28 features each

# Star dataset

Dataset of star properties with their type. We can try to predict the type of each star based in the properties

In [ ]:
# Read the data
# https://www.kaggle.com/datasets/deepu1109/star-dataset/data

stars = pd.read_csv("data/Stars.csv", delimiter=";")

In [ ]:
stars.head()

In [ ]:
stars.boxplot(figsize=(12, 6))
plt.show()

In [ ]:
stars.hist(bins=50, figsize=(16, 10))

plt.tight_layout()
plt.show()

We have two problems: Some values "Temperature" or "Luminosity" are much largers than others: we have to rescale the data. Also, some features ("Color", "Spectral_Class" and "Type", the label) are given in terms of strings of characters. We have to encode them.

Let us use ordinar encoding for the label and one-hot encoding for the features


In [ ]:
# ordinal encoder for one variable (labels)

ordinal = sklearn.preprocessing.LabelEncoder()
ordinal.fit(stars.Type)

In [ ]:
y = ordinal.transform(stars.Type)

In [ ]:
y

In [ ]:
# making a dictionary of types to numbers

types = list(set(stars.Type))
numbers = ordinal.transform(types)

class_names = []

for i in np.argsort(numbers):
    class_names.append(types[i])



In [ ]:
# OneHot encoder for the features "Color" and "Spectral_Class"

ohc = sklearn.preprocessing.OneHotEncoder()

ohc.fit(stars[["Color", "Spectral_Class"]])

In [ ]:
stars[["Color", "Spectral_Class"]]

In [ ]:
encoded_features = ohc.transform(stars[["Color", "Spectral_Class"]])

In [ ]:
X = stars.drop(["Type", "Color", "Spectral_Class"], axis=1).to_numpy()

In [ ]:
X.shape

In [ ]:
X = np.hstack([X, encoded_features.toarray()])

In [ ]:
X.shape

In [ ]:
# define the scaler
scaler = sklearn.preprocessing.StandardScaler()
# train the scaler
scaler.fit(X)

In [ ]:
X = scaler.transform(X)

In [ ]:
# Before we apply any algorithm we must choose our training, validation and test sets.  Work only with training and test

# The module scikit-learn has tools for this purpose

# We use a fraction 75% / 25%

X_train, X_test, y_train, y_test =  sklearn.model_selection.train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# perform diffferent PCAs measuring the explained variance as a function of the number of components

explained = []
components = np.arange(1, 22, 1)

for v in components:
    pca = sklearn.decomposition.PCA(n_components=v)
    pca.fit(X)    

    print(v, pca.explained_variance_ratio_.sum())
    
    explained.append(pca.explained_variance_ratio_.sum())

In [ ]:
# plot the variance explained as a function of the number of components

plt.plot(components, explained, "o-", markersize=7)

plt.xlabel("Number of components")
plt.ylabel("Variance explained")

plt.show()

15 component can be enough

In [ ]:
C = 15
pca = sklearn.decomposition.PCA(n_components=C)
pca.fit(X)

print(f'Total number of components used after PCA : {pca.n_components_}')

pca.explained_variance_ratio_.sum()

In [ ]:
# Transform the train and test sets

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Superconductor dataset

In [ ]:
# features extracted from 21,263 superconductors 
# along with the critical temperature 

# Read de data
# https://www.kaggle.com/datasets/munumbutt/superconductor-dataset/data

superconductors = pd.read_csv("data/superconductor.csv", delimiter=",")

In [ ]:
superconductors.head()

# nothing to encode here

In [ ]:
superconductors.boxplot(figsize=(12, 6))
plt.show()

# High variation, we must rescale

In [ ]:
# correlation

correlations = superconductors.corr()
plt.figure(figsize = (20, 20))
sb.heatmap(correlations, fmt = ".3f", annot = True, cmap = "YlGnBu")
plt.show()

Some pairs of variables are highly correlated, and should be supressed, but we better perform a PCA dimensional reduction.

First, extract the data and rescale

In [ ]:
X = superconductors.drop("critical_temp", axis=1).to_numpy()
y = superconductors["critical_temp"].copy().to_numpy()

In [ ]:
X.shape

In [ ]:
# define the scaler
scaler = sklearn.preprocessing.StandardScaler()

# train the scaler
scaler.fit(X)

# Scale the data

X = scaler.transform(X)

In [ ]:
X.mean()

In [ ]:
# perform diffferent PCAs measuring the explained variance as a function of the number of components

explained = []
components = np.arange(1, 80, 5)

for v in components:
    pca = sklearn.decomposition.PCA(n_components=v)
    pca.fit(X)    

    print(v, pca.explained_variance_ratio_.sum())
    
    explained.append(pca.explained_variance_ratio_.sum())

In [ ]:
# plot the variance explained as a function of the number of components

plt.plot(components, explained, "o-", markersize=7)

plt.xlabel("Number of components")
plt.ylabel("Variance explained")

plt.show()

In [ ]:
# 30 components should be enough

In [ ]:
C = 30
pca = sklearn.decomposition.PCA(n_components=C)
pca.fit(X)

print(f'Total number of components used after PCA : {pca.n_components_}')

pca.explained_variance_ratio_.sum()

In [ ]:
# Choose training and test sets

X_train, X_test, y_train, y_test =  sklearn.model_selection.train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# PCA the train and test sets

# Transform the train and test sets

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)